In [1]:
from pymongo import MongoClient
import json
import pandas as pd
from bson import json_util 
import re
client = MongoClient()

db = client.get_database("euro_2020")


In [2]:
colec = db.teams
colec

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'euro_2020'), 'teams')

In [3]:
archivo = "data/eurocup_2020_results.csv"


In [4]:
#Abriendo el archivo con la funcion open()
with open(archivo) as file: 
    file_data = json.load(file) 
      
if isinstance(file_data, list): 
    colec.insert_many(file_data)   
else: 
    colec.insert_one(file_data)

FileNotFoundError: [Errno 2] No such file or directory: 'data/eurocup_2020_results.csv'

In [5]:
import pandas as pd
from bson import json_util 
import re
import json
from datetime import datetime

c = ['stage', 'date', 'pens', 'pens_home_score', 'pens_away_score',
       'name_home', 'name_away', 'home_score',
       'away_score', 'pos_home', 'pos_away',
       'tshot_h', 'tshot_away', 's_target_home',
       's_target_away', 'won_home', 'won_away',
       'events_list', 'home_p', 'away_p']

df = pd.read_csv("../../data/eurocup_2020_results.csv")

In [6]:
def extract(x,dic):
    for k,v in dic.items():
        if k.lower() in str(x).lower():
            return v
    return int(x)

In [7]:
for c in df.columns:
    if df[c].dtype == object:
        df[c] = df[c].apply(lambda x: x.strip())

In [8]:
df["pens_home_score"] = df["pens_home_score"].apply(extract,dic = {"False":0})
df["pens_away_score"] = df["pens_away_score"].apply(extract,dic = {"False":0})

In [9]:
def extract2(x,dic):
    for k,v in dic.items():
        if k.lower() in str(x).lower():
            return v
    return x

In [10]:
df["stage"] = df["stage"].apply(extract2,dic = {
                                                "Semi-finals":"Semis",
                                                "Quarter-finals": "Quarters",
                                                "Group stage: Matchday 1":"G1",
                                                "Group stage: Matchday 2":"G2",
                                                "Group stage: Matchday 3":"G3"
                                              })

In [11]:

df["date"] = df["date"].apply(lambda x: datetime.strptime(x.replace(".","/").strip(),'%d/%m/%Y'))


In [12]:
cadena = df["events_list"][0]
#"".join(re.findall(r"[a-zA-Z0-9:,\{\}''_-]",cadena))

In [13]:
def sust(x):
    return "".join(re.findall(r"[a-zA-Z0-9:,\{\}''_-]",x))

In [14]:
df["events_list"] = df["events_list"].apply(sust)

In [15]:
df["lineup_home"][0]


"[{'Player_Name': 'Insigne', 'Player_Number': '10'}, {'Player_Name': 'Immobile', 'Player_Number': '17'}, {'Player_Name': 'Chiesa', 'Player_Number': '14'}, {'Player_Name': 'Verratti', 'Player_Number': '6'}, {'Player_Name': 'Jorginho', 'Player_Number': '8'}, {'Player_Name': 'Barella', 'Player_Number': '18'}, {'Player_Name': 'Emerson', 'Player_Number': '13'}, {'Player_Name': 'Chiellini', 'Player_Number': '3'}, {'Player_Name': 'Bonucci', 'Player_Number': '19'}, {'Player_Name': 'Di Lorenzo', 'Player_Number': '2'}, {'Player_Name': 'Donnarumma', 'Player_Number': '21'}]"

In [16]:
df.columns

Index(['stage', 'date', 'pens', 'pens_home_score', 'pens_away_score',
       'team_name_home', 'team_name_away', 'team_home_score',
       'team_away_score', 'possession_home', 'possession_away',
       'total_shots_home', 'total_shots_away', 'shots_on_target_home',
       'shots_on_target_away', 'duels_won_home', 'duels_won_away',
       'events_list', 'lineup_home', 'lineup_away'],
      dtype='object')

In [17]:
def chars(x):
    #specialChars = "[ ']"
    specialChars = "[]"
    for specialChar in specialChars:
        x = x.replace(specialChar, '')
        x = x.replace('"',"")
    
    return x.split(",")

In [18]:
def lineups(lista):
    dic = {}
    for i in range(0,len(lista),2):
        #print(i)
        #aux = {}
        name = lista[i].replace("{'Player_Name': '","").replace("'","")
        num = lista[i+1].replace(" 'Player_Number': '","").replace("'}","")
        #aux["name"] = lista[i].replace("{'Player_Name': '","").replace("'","")
        dic[num] = name
    return dic

In [19]:
lista = df["lineup_home"][0]
dic = {}
for i in range(0,len(lista),2):
    print(i)
    name = lista[i].replace("{'Player_Name': '","").replace("'","")
    num = lista[i+1].replace(" 'Player_Number': '","").replace("'}","")
    dic[num] = name
#dic

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322
324
326
328
330
332
334
336
338
340
342
344
346
348
350
352
354
356
358
360
362
364
366
368
370
372
374
376
378
380
382
384
386
388
390
392
394
396
398
400
402
404
406
408
410
412
414
416
418
420
422
424
426
428
430
432
434
436
438
440
442
444
446
448
450
452
454
456
458
460
462
464
466
468
470
472
474
476
478
480
482
484
486
488
490
492
494
496
498
500
502
504
506
508
510
512
514
516
518
520
522
524
526

IndexError: string index out of range

In [20]:
df["lineup_home"] = df["lineup_home"].apply(chars)
df["lineup_home"] = df["lineup_home"].apply(lineups)

In [21]:
df["lineup_away"] = df["lineup_away"].apply(chars)
df["lineup_away"] = df["lineup_away"].apply(lineups)

In [22]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,2021-07-11,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"{'event_team':'away','event_time':2','event_ty...","{'10': 'Insigne', '17': ' Immobile', '14': ' C...","{'9': 'Kane', '19': ' Mount', '10': ' Sterling..."
1,Semis,2021-07-07,False,0,0,England,Denmark,2,1,59%,41%,20,6,10,3,50%,50%,"{'event_team':'away','event_time':30','event_t...","{'9': 'Kane', '10': ' Sterling', '19': ' Mount...","{'14': 'Krogh Damsgaard', '12': ' Dolberg', '9..."
2,Semis,2021-07-06,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"{'event_team':'away','event_time':51','event_t...","{'10': 'Insigne', '17': ' Immobile', '14': ' C...","{'11': 'Torres', '19': ' Olmo Carvajal', '21':..."
3,Quarters,2021-07-03,False,0,0,Ukraine,England,0,4,48%,52%,7,10,2,6,42%,59%,"{'event_team':'away','event_time':4','event_ty...","{'9': 'Yaremchuk', '7': ' Yarmolenko', '16': '...","{'9': 'Kane', '10': ' Sterling', '19': ' Mount..."
4,Quarters,2021-07-03,False,0,0,Czech Republic,Denmark,1,2,56%,44%,16,11,5,7,48%,52%,"{'event_team':'away','event_time':5','event_ty...","{'10': 'Schick', '13': ' Ševčík', '7': ' Barák...","{'12': 'Dolberg', '9': ' Braithwaite', '14': '..."
5,Quarters,2021-07-02,False,0,0,Belgium,Italy,1,2,46%,54%,10,14,4,3,58%,42%,"{'event_team':'away','event_time':13','event_t...","{'9': 'Lukaku', '25': ' Doku', '7': ' De Bruyn...","{'10': 'Insigne', '17': ' Immobile', '14': ' C..."
6,Quarters,2021-07-02,True,1,3,Switzerland,Spain,1,1,27%,73%,8,28,2,10,58%,42%,"{'event_team':'away','event_time':8','event_ty...","{'9': 'Seferovic', '14': ' Zuber', '23': ' Sha...","{'22': 'Sarabia', '7': ' Morata', '11': ' Torr..."
7,Round of 16,2021-06-29,False,0,0,Sweden,Ukraine,1,2,45%,55%,13,15,3,4,48%,52%,"{'event_team':'away','event_time':27','event_t...","{'11': 'Isak', '21': ' Kulusevski', '10': ' Fo...","{'9': 'Yaremchuk', '7': ' Yarmolenko', '17': '..."
8,Round of 16,2021-06-29,False,0,0,England,Germany,2,0,45%,55%,5,9,4,3,55%,45%,"{'event_team':'home','event_time':8','event_ty...","{'10': 'Sterling', '9': ' Kane', '25': ' Saka'...","{'11': 'Werner', '25': ' Müller', '7': ' Haver..."
9,Round of 16,2021-06-28,True,4,5,France,Switzerland,3,3,55%,45%,26,12,8,5,55%,45%,"{'event_team':'away','event_time':15','event_t...","{'10': 'Mbappe', '19': ' Benzema', '14': ' Rab...","{'9': 'Seferovic', '7': ' Embolo', '23': ' Sha..."


In [23]:
#df.groupby(["team_name_home"]).sum()

In [24]:
g_home = df.groupby(["team_name_home"]).sum().reset_index()[["team_name_home","team_home_score"]]
g_away = df.groupby(["team_name_away"]).sum().reset_index()[["team_name_away","team_away_score"]]

new = pd.DataFrame([g_away["team_name_away"],g_away["team_away_score"],g_home["team_home_score"]])
new = new.T
new["tot"]= new["team_home_score"]+new["team_away_score"]
new.rename(columns={'team_home_score': 'home', 
                       'team_away_score': 'away',
                       'team_name_away': 'team'}, inplace=True)
#new

In [25]:
selected_team = ["Croatia"]

new = new[new['team'].isin(selected_team)]
new
nuevodf = new.T

nuevodf = nuevodf.drop(["tot"],axis=0)
nuevodf.columns[0]


2

In [26]:
nuevodf.rename(columns = columnas)
#nuevodf = nuevodf.drop(["team"],axis=0)
nuevodf

NameError: name 'columnas' is not defined

In [27]:
nuevodf[4]

KeyError: 4

In [28]:
db
#vaciar collection
res = db.matchs.delete_many({})
res

In [29]:
db.matchs.insert_many(df.to_dict('records'))

In [30]:
df1 = pd.read_csv("../../data/eurocup_2020_formated.csv")
df1

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[{'event_team': 'away', 'event_time': "" 2' "", ...","{'10': 'Insigne', '17': ' Immobile', '14': ' C...","{'9': 'Kane', '19': ' Mount', '10': ' Sterling..."
1,Semi-finals,07.07.2021,False,0,0,England,Denmark,2,1,59%,41%,20,6,10,3,50%,50%,"[{'event_team': 'away', 'event_time': "" 30' "",...","{'9': 'Kane', '10': ' Sterling', '19': ' Mount...","{'14': 'Krogh Damsgaard', '12': ' Dolberg', '9..."
2,Semi-finals,06.07.2021,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"[{'event_team': 'away', 'event_time': "" 51' "",...","{'10': 'Insigne', '17': ' Immobile', '14': ' C...","{'11': 'Torres', '19': ' Olmo Carvajal', '21':..."
3,Quarter-finals,03.07.2021,False,0,0,Ukraine,England,0,4,48%,52%,7,10,2,6,42%,59%,"[{'event_team': 'away', 'event_time': "" 4' "", ...","{'9': 'Yaremchuk', '7': ' Yarmolenko', '16': '...","{'9': 'Kane', '10': ' Sterling', '19': ' Mount..."
4,Quarter-finals,03.07.2021,False,0,0,Czech Republic,Denmark,1,2,56%,44%,16,11,5,7,48%,52%,"[{'event_team': 'away', 'event_time': "" 5' "", ...","{'10': 'Schick', '13': ' Ševčík', '7': ' Barák...","{'12': 'Dolberg', '9': ' Braithwaite', '14': '..."
5,Quarter-finals,02.07.2021,False,0,0,Belgium,Italy,1,2,46%,54%,10,14,4,3,58%,42%,"[{'event_team': 'away', 'event_time': "" 13' "",...","{'9': 'Lukaku', '25': ' Doku', '7': ' De Bruyn...","{'10': 'Insigne', '17': ' Immobile', '14': ' C..."
6,Quarter-finals,02.07.2021,True,1,3,Switzerland,Spain,1,1,27%,73%,8,28,2,10,58%,42%,"[{'event_team': 'away', 'event_time': "" 8' "", ...","{'9': 'Seferovic', '14': ' Zuber', '23': ' Sha...","{'22': 'Sarabia', '7': ' Morata', '11': ' Torr..."
7,Round of 16,29.06.2021,False,0,0,Sweden,Ukraine,1,2,45%,55%,13,15,3,4,48%,52%,"[{'event_team': 'away', 'event_time': "" 27' "",...","{'11': 'Isak', '21': ' Kulusevski', '10': ' Fo...","{'9': 'Yaremchuk', '7': ' Yarmolenko', '17': '..."
8,Round of 16,29.06.2021,False,0,0,England,Germany,2,0,45%,55%,5,9,4,3,55%,45%,"[{'event_team': 'home', 'event_time': "" 8' "", ...","{'10': 'Sterling', '9': ' Kane', '25': ' Saka'...","{'11': 'Werner', '25': ' Müller', '7': ' Haver..."
9,Round of 16,28.06.2021,True,4,5,France,Switzerland,3,3,55%,45%,26,12,8,5,55%,45%,"[{'event_team': 'away', 'event_time': "" 15' "",...","{'10': 'Mbappe', '19': ' Benzema', '14': ' Rab...","{'9': 'Seferovic', '7': ' Embolo', '23': ' Sha..."


In [ ]:
from pymongo import MongoClient
from dotenv import load_dotenv
import os
load_dotenv()

password = os.getenv("MONGO_PASS")

url=f"mongodb+srv://euro2020:<{password}>@cluster0.bk0gi.mongodb.net/myFirstDatabase?retryWrites=true&w=majority"

client = MongoClient(url)


